In [ ]:
import os
home = os.path.expanduser("~")
with open(os.path.join(home, 'name.txt')) as f:
    server_name = f.readline().strip()
server_name

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-o", "--organ", help="Organ")
parser.add_argument("-g", "--gpu", help="GPU", type=int)
args = parser.parse_args()

organ = args.organ
gpu = args.gpu

print(args)

# Hyperparameters

In [ ]:
MODEL_NAME = 'scribble'

dropout = 0. # drop rate
weight_decay  = 0. # weight decay of the optimizer
epochs = 1000 # number of total epochs

# we'll save the models of the `number_of_saved_models` last epochs.
number_of_saved_models = 20

# the depth of the model
model_depth = 4

batch_size = 1

# optimizer
optim_name = 'adam'
learning_rate = 1e-1
learning_rate_decay = (100, 0.5)

# scribble settings
scribble_size_shape = 'circle'
scribble_bounds = 'dynamic'
scribble_mode = '2d'
scribble_cweight = 1e-2

# Parameters

In [ ]:
if server_name == '78':
    data_path = '/media/external_3TB/home/mnikdan/visceral/data/'
    results_path = '/media/external_3TB/home/mnikdan/visceral/results/'
else:
    data_path = '../visceral-data/data/'
    results_path = '../visceral-data/results/'

In [ ]:
import os

# path to the preprocessed data folder
organ_data_path = os.path.join(data_path, organ)

if not os.path.exists(organ_data_path):
    print('retrieving {} data from the main server...'.format(organ))
    
    # this line was edited before submission to MILA
    command = 'scp -r mnikdan@xxx.xxx.xxx.xxx:visceral-data/data/{} {}'.format(
        organ,
        data_path
    )
    os.system(command)

# Imports

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu) # which gpu to use

import numpy as np
import copy
from tqdm import tqdm
import pickle

import matplotlib
import matplotlib.pyplot as plt

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader

from utils.model import UNet3D
from utils.metrics import dice_coef
from utils.data import read_data
from utils.scribble import AutoScribbleDataset, scribble_loss
from utils.constants import image_slices, image_shape
from bot.send import send_to_bot

np.random.seed(0)

result_folder_name = 'visceral_{}-{}-epochs_{}-server_{}'.format(organ, MODEL_NAME, epochs, server_name)

print(result_folder_name)

# Device

In [ ]:
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Loading the data

In [ ]:
def read_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [ ]:
train_val_patients, train_val_X, train_val_y, train_val_real_slices = read_pickle(os.path.join(organ_data_path, 'train_val'))
test_patients, test_X, test_y, test_real_slices = read_pickle(os.path.join(organ_data_path, 'test'))

In [ ]:
def split_train_val(X):
    return X[:-15], X[-15:]

train_patients, val_patients = split_train_val(train_val_patients)
train_X, val_X = split_train_val(train_val_X)
train_y, val_y = split_train_val(train_val_y)
train_real_slices, val_real_slices = split_train_val(train_val_real_slices)

# Define DataLoader

In [ ]:
train_loader = DataLoader(
    AutoScribbleDataset(
        train_X,
        train_y,
        train_real_slices,
        scribble_size_shape=scribble_size_shape,
        save_path=organ_data_path,
        is_double=False
    ),
    batch_size=batch_size,
    shuffle=True,
    num_workers=6
)

# Model

In [ ]:
# initialize the model
model = UNet3D(input_dim=1, output_dim=1, depth=model_depth, dropout=dropout).to(device).float()

# define the optimizer.
optimizer = None
if optim_name == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
elif optim_name == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

scheduler = None
if learning_rate_decay is not None:
    step_size, gamma = learning_rate_decay
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size, gamma)

# Train

In [ ]:
def dice(y_pred, y_true):
    assert np.sum(y_true == 2) == 0

    inter = np.sum(y_pred * y_true)
    union = np.sum(y_pred) + np.sum(y_true)
    if union == 0:
        union = 1
    return 2 * inter / union

def test(model, ttest_X, ttest_y, ttest_real_slices):
    model.eval()
    running_test_dice = 0
    with torch.no_grad():
        for i in range(len(ttest_X)):
            x = torch.Tensor(ttest_X[i:i+1]).to(device).float()
            y = ttest_y[i:i+1]
            
            o = model(x)
            
            real_slices = ttest_real_slices[i]
            real_o = o[:, :, :real_slices, :, :]
            real_y = y[:, :, :real_slices, :, :]
            
            running_test_dice += dice(
                torch.round(torch.sigmoid(real_o)).cpu().detach().numpy(), 
                real_y
            )
            
    model.train()
    return running_test_dice / len(ttest_X)

In [ ]:
# create a folder to store the results
if not os.path.isdir(results_path):
    os.mkdir(results_path)
os.mkdir(os.path.join(results_path, result_folder_name)) # we store the results of the training in this folder

def log(l):
    with open(os.path.join(results_path, result_folder_name, 'log.txt'), 'a') as log_f:
        log_f.write("%s\n" % l)
    print(l)

In [ ]:
samples_count = len(train_X)

best_val_dice = 0
best_epoch = 0
best_model_state = None

for epoch in range(epochs):
    running_hloss = 0.
    running_closs = 0.
    running_loss = 0.
    running_real_dice = 0.
    
    for i, data in tqdm(enumerate(train_loader), total=train_loader.__len__()): # iterate over batches

        X, Y, index = data
        
        X, Y = X.to(device).float(), Y.to(device).float()
        
        optimizer.zero_grad()
        
        outputs = model(X)
        
        real_slices = train_real_slices[index]
        real_outputs = outputs[:, :, :real_slices, :, :]
        real_Y = Y[:, :, :real_slices, :, :]
        real_full_y = train_y[index:index+1, :, :real_slices, :, :]
        
        hloss, closs, loss = scribble_loss(
            real_outputs,
            real_Y,
            real_full_y,
            scribble_bounds,
            scribble_mode,
            scribble_cweight
        )
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        running_hloss += hloss.item()
        running_closs += closs.item()
        
        running_real_dice += dice(
            torch.round(torch.sigmoid(real_outputs)).cpu().detach().numpy(), 
            real_full_y
        )
        
    if scheduler is not None:
        scheduler.step()
    
    # if we are on the last `number_of_saved_models` epochs, save the model.
    if epochs - epoch <= number_of_saved_models or epoch % 100 == 99:
        # save the model in the result folder with name 'model-' + epoch.
        torch.save(model.state_dict(), os.path.join(results_path, result_folder_name, 'model-' + str(epoch + 1)))
    
    val_dice = test(model, val_X, val_y, val_real_slices)
    test_dice = test(model, test_X, test_y, test_real_slices)
    
    if val_dice >= best_val_dice:
        best_val_dice = val_dice
        best_model_state = copy.deepcopy(model.state_dict())
        best_epoch = epoch + 1
        torch.save(best_model_state, os.path.join(results_path, result_folder_name, 'best-model'))
    
    # log and print the losses and dices of this epoch.
    log('Epoch %d: loss=%.6f, hloss=%.6f, closs=%.6f, real_dice=%.4f, val_dice=%.4f, test_dice=%.4f' %
               (epoch + 1,
                running_loss / samples_count,
                running_hloss / samples_count,
                running_closs / samples_count,
                running_real_dice / samples_count,
                val_dice,
                test_dice))
log('Best Epoch: {}'.format(best_epoch))
print('Finished Training')

In [ ]:
print('Result save in: {}'.format(result_folder_name))
send_to_bot(os.path.join(results_path, result_folder_name), result_folder_name)